In [8]:
# !pip install textblob
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_

In [14]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kriks.iitk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [74]:
import pandas as pd
import enchant
from textblob import TextBlob,Word
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()
from __future__ import division
import re
import math

In [103]:
def split_in_chunks(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    if len(chunks) > 1 and len(chunks[-1]) < 10:
        chunks[-2] += chunks[-1]
        chunks.pop(-1)
    return chunks


def unique_chars_per_chunk_percentage(text, chunk_size):
    chunks = split_in_chunks(text, chunk_size)
    unique_chars_percentages = []
    for chunk in chunks:
        total = len(chunk)
        unique = len(set(chunk))
        unique_chars_percentages.append(unique / total)
    return sum(unique_chars_percentages) / len(unique_chars_percentages) * 100


def vowels_percentage(text):
    vowels = 0
    total = 0
    for c in text:
        if not c.isalpha():
            continue
        total += 1
        if c in "aeiouAEIOU":
            vowels += 1
    if total != 0:
        return vowels / total * 100
    else:
        return 0


def word_to_char_ratio(text):
    chars = len(text)
    words = len([x for x in re.split(r"[\W_]", text) if x.strip() != ""])
    return words / chars * 100


def deviation_score(percentage, lower_bound, upper_bound):
    if percentage < lower_bound:
        return math.log(lower_bound - percentage, lower_bound) * 100
    elif percentage > upper_bound:
        return math.log(percentage - upper_bound, 100 - upper_bound) * 100
    else:
        return 0


def classify(text):
    if text is None or len(text) == 0:
        return 0.0
    ucpcp = unique_chars_per_chunk_percentage(text, 35)
    vp = vowels_percentage(text)
    wtcr = word_to_char_ratio(text)

    ucpcp_dev = max(deviation_score(ucpcp, 45, 50), 1)
    vp_dev = max(deviation_score(vp, 35, 45), 1)
    wtcr_dev = max(deviation_score(wtcr, 15, 20), 1)

    return max((math.log10(ucpcp_dev) + math.log10(vp_dev) +
                math.log10(wtcr_dev)) / 6 * 100, 1)

In [104]:
file = 'merlin_feedback_tagging.xlsx'
feedback_tag_df = pd.read_excel(file,sheet_name = 1)
feedback_wo_tag = pd.read_excel(file,sheet_name = 0)
ft_df = feedback_tag_df[['translated','tags']]

In [105]:
def is_not_gibberish(text):
    blob_object = TextBlob(text)
    for wrd in blob_object.words:
        chk_wrd = Word(wrd).lemmatize()
        chk_wrd = Word(wrd).lemmatize('v')
        print(chk_wrd)
        if chk_wrd in words.words():
            continue
        else:
            return False
    return True 

In [106]:
ft_df['translated'].apply(lambda x: ''.join(TextBlob(str(x)).correct()))
# ft_df['correct_feed'] = gibberish_filter(str(x['translated']))

0                                      CTR+M did not work
1                        again and again asking for login
2       I just still don't use Vein because this don't...
3                                shortest doesn´t working
4       Your expansion stopped working, apparently due...
                              ...                        
1241                                              privacy
1242                                           not a free
1243                                         too invasion
1244                    did not provide effective results
1245    not supported Ukrainian, English I do not unde...
Name: translated, Length: 1246, dtype: object

In [108]:
ft_df

,translated,tags
0,CTR+M did not work,SC
1,again and again asking for login,bug
2,I just still don't use Melin because this don'...,browser
3,shortcut doesn´t working,SC
4,"Your expansion stopped working, apparently due...",??
5,ADO shortcut kicks in when user types a key su...,bug
6,It doesn't work in Brave,browser
7,Just didn't work. Didn't display on Google Sea...,bug
8,it crashes opera,bug
9,The way the app pops up when I am searching fo...,maps


In [107]:
# new_ft_df = ft_df.assign(correct_feed = lambda x: is_not_gibberish(x['translated']))
ft_df['is_gibber'] = ft_df.apply(lambda x: classify((str(x['translated']))))

KeyError: ('translated', 'occurred at index translated')

In [87]:
ft_df.head()

,translated,tags,correct_feedback,correct_feed,axis,is_not_gibber
0,CTR+M did not work,SC,False,False,1,False
1,again and again asking for login,bug,False,False,1,True
2,I just still don't use Melin because this don'...,browser,False,False,1,False
3,shortcut doesn´t working,SC,False,False,1,False
4,"Your expansion stopped working, apparently due...",??,False,False,1,False


In [95]:
text_ch = "shortcut doesn working"
bl = TextBlob(text_ch)
print(bl.correct())
# is_not_gibberish(text_ch)

shortest doesn working


In [14]:
lemmatizer.lemmatize('birds')

'bird'

In [41]:
ft_df = feedback_tag_df[['date','email','response','translated','tags']]
ft_df = ft_df[['translated','tags']]
ft_df['translated'].apply(lambda x: ''.join(TextBlob(str(x)).correct()))

0                                      CTR+M did not work
1                        again and again asking for login
2       I just still don't use Vein because this don't...
3                                shortest doesn´t working
4       Your expansion stopped working, apparently due...
                              ...                        
1241                                              privacy
1242                                           not a free
1243                                         too invasion
1244                    did not provide effective results
1245    not supported Ukrainian, English I do not unde...
Name: translated, Length: 1246, dtype: object

In [8]:
text = 'again and again asking for login'
print(tb(text).correct())

again and again asking for login


In [42]:
ft_df.head()

,translated,tags
0,CTR+M did not work,SC
1,again and again asking for login,bug
2,I just still don't use Melin because this don'...,browser
3,shortcut doesn´t working,SC
4,"Your expansion stopped working, apparently due...",??


In [32]:
values_list = [['hey how are u and wat u doing'], ['Hey wats up'], ['Ahhh u are gud'],
               ['what u doing'], ['Hey how are you doing'], ['for the sake'],
               ['u got it']]
 
# creating a pandas dataframe
df_test = pd.DataFrame(values_list, columns=['test_str'])

,test_str
0,hey how are u and wat u doing
1,Hey wats up
2,Ahhh u are gud
3,what u doing
4,Hey how are you doing
5,for the sake
6,u got it


In [36]:
df_test['test_str'].apply(lambda x: ''.join(TextBlob(str(x)).correct()))

0    hey how are u and wat u doing
1                       Key was up
2                    Heh u are god
3                     what u doing
4            Key how are you doing
5                     for the sake
6                         u got it
Name: test_str, dtype: object

In [34]:
df_test

,test_str
0,hey how are u and wat u doing
1,Hey wats up
2,Ahhh u are gud
3,what u doing
4,Hey how are you doing
5,for the sake
6,u got it
